<a href="https://colab.research.google.com/github/gubertoli/flower/blob/baselines_fedavgm/FedAvgM_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FedAvgM Baseline

Reference:

- **FedAvgM paper**: Hsu, T. M. H., Qi, H., & Brown, M. (2019). Measuring the effects of non-identical data distribution for federated visual classification. arXiv preprint arXiv:1909.06335. [Link](https://arxiv.org/abs/1909.06335)


---

In [ ]:
!pip install -q flwr["simulation"]

In [ ]:
import os

# Make TensorFlow logs less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import math
import tensorflow as tf
import flwr as fl
import numpy as np
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

In [ ]:
print(fl.__version__)

1.4.0


## Simulation Parameters
### Variables
- **FEMNIST**: Boolean variable, if set True will run simulation with the FEMNIST dataset, if False runs with CIFAR-10
- **CONCENTRATION**: Represents the 8 possible values used in the FedAvgM paper (referred as "α" alpha), this concentration parameter is used by the Direchlet distribution to create non-IID distributions between the clients. According to Figure 2 from the paper, the applicable values are [100., 10., 1., 0.5, 0.2, 0.1, 0.05, 0.0]. The understanding for this variable is that α→∞ all clients have identical distribution, and α→𝟢 each client hold samples from only one class.
- **REPORTING_FRACTION**: It is the % of clients participating in the federated learning process, in the paper it is referred as "C" and have the values [0.05, 0.1, 0.4]


### Constants
- **NUM_CLIENTS**: Set to 100
- **COMMUNICATION_ROUNDS**: 10000
- **MOMENTUM**: 0.9 - According to the FedAvgM paper could assume the values [0, 0.7, 0.9, 0.97, 0.99, 0.997], but does not change in this evaluation/simulation
- **LEARNING_RATE**: The learning rate for the FedAvgM aggregation algorithm (server-side), it is constant and 1.0
- **BATCH SIZE**: referred as "B" in the paper, does not change in this evaluation, with the value 64

In [ ]:
FEMNIST = False     # If False, FedAvgM runs with CIFAR-10
NUM_CLIENTS = 100   # The paper runs with 100 clients, does not change in this evaluation
CONCENTRATION = 1.  # The papers uses 8 values (referred as "α") / From Figure 2: [100., 10., 1., 0.5, 0.2, 0.1, 0.05, 0.0]

# Server-side parameters
REPORTING_FRACTION = 0.05     # [0.05, 0.1, 0.4]   # referred as "C" in the paper
COMMUNICATION_ROUNDS = 10000  # does not change in this evaluation
MOMENTUM = 0.9                # [0, 0.7, 0.9, 0.97, 0.99, 0.997] - does not change in this evaluation
LEARNING_RATE = 1.0           # does not change in this evaluation

# Client-side parameters
BATCH_SIZE = 64     # referred as "B" in the paper, does not change in this evaluation
LOCAL_EPOCHS = 1    # the paper uses local epoch of 1 or 5 (referred as "E")

In [ ]:
# From Flower common functions

from typing import List, Optional, Tuple, Union
from numpy.random import BitGenerator, Generator, SeedSequence

XY = Tuple[np.ndarray, np.ndarray]
XYList = List[XY]
PartitionedDataset = Tuple[XYList, XYList]

np.random.seed(2020)

def shuffle(x: np.ndarray, y: np.ndarray) -> XY:
    """Shuffle x and y."""
    idx = np.random.permutation(len(x))
    return x[idx], y[idx]


def sort_by_label(x: np.ndarray, y: np.ndarray) -> XY:
    """Sort by label.

    Assuming two labels and four examples the resulting label order
    would be 1,1,2,2
    """
    idx = np.argsort(y, axis=0).reshape((y.shape[0]))
    return (x[idx], y[idx])

def split_array_at_indices(
    x: np.ndarray, split_idx: np.ndarray
) -> List[List[np.ndarray]]:
    """Splits an array `x` into list of elements using starting indices from
    `split_idx`.

        This function should be used with `unique_indices` from `np.unique()` after
        sorting by label.

    Args:
        x (np.ndarray): Original array of dimension (N,a,b,c,...)
        split_idx (np.ndarray): 1-D array contaning increasing number of
            indices to be used as partitions. Initial value must be zero. Last value
            must be less than N.

    Returns:
        List[List[np.ndarray]]: List of list of samples.
    """

    if split_idx.ndim != 1:
        raise ValueError("Variable `split_idx` must be a 1-D numpy array.")
    if split_idx.dtype != np.int64:
        raise ValueError("Variable `split_idx` must be of type np.int64.")
    if split_idx[0] != 0:
        raise ValueError("First value of `split_idx` must be 0.")
    if split_idx[-1] >= x.shape[0]:
        raise ValueError(
            """Last value in `split_idx` must be less than
            the number of samples in `x`."""
        )
    if not np.all(split_idx[:-1] <= split_idx[1:]):
        raise ValueError("Items in `split_idx` must be in increasing order.")

    num_splits: int = len(split_idx)
    split_idx = np.append(split_idx, x.shape[0])

    list_samples_split: List[List[np.ndarray]] = [[] for _ in range(num_splits)]
    for j in range(num_splits):
        tmp_x = x[split_idx[j] : split_idx[j + 1]]  # noqa: E203
        for sample in tmp_x:
            list_samples_split[j].append(sample)

    return list_samples_split


def sample_without_replacement(
    distribution: np.ndarray,
    list_samples: List[List[np.ndarray]],
    num_samples: int,
    empty_classes: List[bool],
) -> Tuple[XY, List[bool]]:
    """Samples from a list without replacement using a given distribution.

    Args:
        distribution (np.ndarray): Distribution used for sampling.
        list_samples(List[List[np.ndarray]]): List of samples.
        num_samples (int): Total number of items to be sampled.
        empty_classes (List[bool]): List of booleans indicating which classes are empty.
            This is useful to differentiate which classes should still be sampled.

    Returns:
        XY: Dataset contaning samples
        List[bool]: empty_classes.
    """
    if np.sum([len(x) for x in list_samples]) < num_samples:
        raise ValueError(
            """Number of samples in `list_samples` is less than `num_samples`"""
        )

    # Make sure empty classes are not sampled
    # and solves for rare cases where
    if not empty_classes:
        empty_classes = len(distribution) * [False]

    distribution = exclude_classes_and_normalize(
        distribution=distribution, exclude_dims=empty_classes
    )

    data: List[np.ndarray] = []
    target: List[np.ndarray] = []

    for _ in range(num_samples):
        sample_class = np.where(np.random.multinomial(1, distribution) == 1)[0][0]
        sample: np.ndarray = list_samples[sample_class].pop()

        data.append(sample)
        target.append(sample_class)

        # If last sample of the class was drawn, then set the
        #  probability density function (PDF) to zero for that class.
        if len(list_samples[sample_class]) == 0:
            empty_classes[sample_class] = True
            # Be careful to distinguish between classes that had zero probability
            # and classes that are now empty
            distribution = exclude_classes_and_normalize(
                distribution=distribution, exclude_dims=empty_classes
            )
    data_array: np.ndarray = np.concatenate([data], axis=0)
    target_array: np.ndarray = np.array(target, dtype=np.int64)

    return (data_array, target_array), empty_classes


def exclude_classes_and_normalize(
    distribution: np.ndarray, exclude_dims: List[bool], eps: float = 1e-5
) -> np.ndarray:
    """Excludes classes from a distribution.

    This function is particularly useful when sampling without replacement.
    Classes for which no sample is available have their probabilities are set to 0.
    Classes that had probabilities originally set to 0 are incremented with
     `eps` to allow sampling from remaining items.

    Args:
        distribution (np.array): Distribution being used.
        exclude_dims (List[bool]): Dimensions to be excluded.
        eps (float, optional): Small value to be addad to non-excluded dimensions.
            Defaults to 1e-5.

    Returns:
        np.ndarray: Normalized distributions.
    """
    if np.any(distribution < 0) or (not np.isclose(np.sum(distribution), 1.0)):
        raise ValueError("distribution must sum to 1 and have only positive values.")

    if distribution.size != len(exclude_dims):
        raise ValueError(
            """Length of distribution must be equal
            to the length `exclude_dims`."""
        )
    if eps < 0:
        raise ValueError("""The value of `eps` must be positive and small.""")

    distribution[[not x for x in exclude_dims]] += eps
    distribution[exclude_dims] = 0.0
    sum_rows = np.sum(distribution) + np.finfo(float).eps
    distribution = distribution / sum_rows

    return distribution


def create_lda_partitions(
    dataset: XY,
    dirichlet_dist: Optional[np.ndarray] = None,
    num_partitions: int = 100,
    concentration: Union[float, np.ndarray, List[float]] = 0.5,
    accept_imbalanced: bool = False,
    seed: Optional[Union[int, SeedSequence, BitGenerator, Generator]] = None,
) -> Tuple[XYList, np.ndarray]:
    """Create imbalanced non-iid partitions using Latent Dirichlet Allocation
    (LDA) without resampling.

    Args:
        dataset (XY): Dataset containing samples X and labels Y.
        dirichlet_dist (numpy.ndarray, optional): previously generated distribution to
            be used. This is useful when applying the same distribution for train and
            validation sets.
        num_partitions (int, optional): Number of partitions to be created.
            Defaults to 100.
        concentration (float, np.ndarray, List[float]): Dirichlet Concentration
            (:math:`\\alpha`) parameter. Set to float('inf') to get uniform partitions.
            An :math:`\\alpha \\to \\Inf` generates uniform distributions over classes.
            An :math:`\\alpha \\to 0.0` generates one class per client. Defaults to 0.5.
        accept_imbalanced (bool): Whether or not to accept imbalanced output classes.
            Default False.
        seed (None, int, SeedSequence, BitGenerator, Generator):
            A seed to initialize the BitGenerator for generating the Dirichlet
            distribution. This is defined in Numpy's official documentation as follows:
            If None, then fresh, unpredictable entropy will be pulled from the OS.
            One may also pass in a SeedSequence instance.
            Additionally, when passed a BitGenerator, it will be wrapped by Generator.
            If passed a Generator, it will be returned unaltered.
            See official Numpy Documentation for further details.

    Returns:
        Tuple[XYList, numpy.ndarray]: List of XYList containing partitions
            for each dataset and the dirichlet probability density functions.
    """
    # pylint: disable=too-many-arguments,too-many-locals

    x, y = dataset
    x, y = shuffle(x, y)
    x, y = sort_by_label(x, y)

    if (x.shape[0] % num_partitions) and (not accept_imbalanced):
        raise ValueError(
            """Total number of samples must be a multiple of `num_partitions`.
               If imbalanced classes are allowed, set
               `accept_imbalanced=True`."""
        )

    num_samples = num_partitions * [0]
    for j in range(x.shape[0]):
        num_samples[j % num_partitions] += 1

    # Get number of classes and verify if they matching with
    classes, start_indices = np.unique(y, return_index=True)

    # Make sure that concentration is np.array and
    # check if concentration is appropriate
    concentration = np.asarray(concentration)

    # Check if concentration is Inf, if so create uniform partitions
    partitions: List[XY] = [(_, _) for _ in range(num_partitions)]
    if float("inf") in concentration:
        partitions = create_partitions(
            unpartitioned_dataset=(x, y),
            iid_fraction=1.0,
            num_partitions=num_partitions,
        )
        dirichlet_dist = get_partitions_distributions(partitions)[0]

        return partitions, dirichlet_dist

    if concentration.size == 1:
        concentration = np.repeat(concentration, classes.size)
    elif concentration.size != classes.size:  # Sequence
        raise ValueError(
            f"The size of the provided concentration ({concentration.size}) ",
            f"must be either 1 or equal number of classes {classes.size})",
        )

    # Split into list of list of samples per class
    list_samples_per_class: List[List[np.ndarray]] = split_array_at_indices(
        x, start_indices
    )

    if dirichlet_dist is None:
        dirichlet_dist = np.random.default_rng(seed).dirichlet(
            alpha=concentration, size=num_partitions
        )

    if dirichlet_dist.size != 0:
        if dirichlet_dist.shape != (num_partitions, classes.size):
            raise ValueError(
                f"""The shape of the provided dirichlet distribution
                 ({dirichlet_dist.shape}) must match the provided number
                  of partitions and classes ({num_partitions},{classes.size})"""
            )

    # Assuming balanced distribution
    empty_classes = classes.size * [False]
    for partition_id in range(num_partitions):
        partitions[partition_id], empty_classes = sample_without_replacement(
            distribution=dirichlet_dist[partition_id].copy(),
            list_samples=list_samples_per_class,
            num_samples=num_samples[partition_id],
            empty_classes=empty_classes,
        )

    return partitions, dirichlet_dist

In [ ]:
if FEMNIST == True:
  (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
  x_train = x_train.astype('float32') / 255
  x_test = x_test.astype('float32') / 255
  input_shape = x_train.shape[1:]
  num_classes = len(np.unique(y_train))
else:
  (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
  x_train = x_train.astype('float32') / 255
  x_test = x_test.astype('float32') / 255
  input_shape = x_train.shape[1:]
  num_classes = len(np.unique(y_train))

In [20]:
dataset = [x_train, y_train]
def partition(concentration=CONCENTRATION):
  partitions, b = create_lda_partitions(dataset, num_partitions=NUM_CLIENTS, concentration=concentration * num_classes, seed=1234)
  return partitions


In [33]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_val, y_val) -> None:
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, epochs=LOCAL_EPOCHS, verbose=2)
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, acc = self.model.evaluate(self.x_val, self.y_val, verbose=2)
        return loss, len(self.x_val), {"accuracy": acc}

The **FedAvgM** paper uses the same CNN model presented in the following paper:

- McMahan, B., Moore, E., Ramage, D., Hampson, S., & y Arcas, B. A. (2017, April). Communication-efficient learning of deep networks from decentralized data. In Artificial intelligence and statistics (pp. 1273-1282). PMLR. ([Link](http://proceedings.mlr.press/v54/mcmahan17a/mcmahan17a.pdf)):

As the following excerpt:

"*A CNN with two 5x5 convolution layers (the first with 32 channels, the second with 64, each followed with 2x2 max pooling), a fully connected layer with 512 units and ReLu activation, and a final softmax output layer (1,663,370
total parameters)"*

<font color="red">However, this architecture implemented below results in 878,538 parameters.</font>




In [22]:
model = keras.Sequential([
  keras.layers.Conv2D(32, (5,5), activation='relu', input_shape=input_shape),
  keras.layers.MaxPooling2D(2,2),
  keras.layers.Conv2D(64, (5,5), activation='relu'),
  keras.layers.MaxPooling2D(2,2),
  keras.layers.Flatten(),
  keras.layers.Dense(512, activation='relu'),
  keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 32)        2432      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 64)        51264     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1600)              0         
                                                                 
 dense_2 (Dense)             (None, 512)              

In [26]:
def client_fn(cid: str) -> fl.client.Client:
  partitions = partition(CONCENTRATION)
  full_x_train_cid, full_y_train_cid = partitions[int(cid)]
  full_y_train_cid = to_categorical(full_y_train_cid, num_classes=num_classes)

  # Use 10% of the client's training data for validation
  split_idx = math.floor(len(full_x_train_cid) * 0.9)
  x_train_cid, y_train_cid = (
    full_x_train_cid[:split_idx],
    full_y_train_cid[:split_idx],
  )
  x_val_cid, y_val_cid = full_x_train_cid[split_idx:], full_y_train_cid[split_idx:]


  return FlowerClient(model, x_train_cid, y_train_cid, x_val_cid, y_val_cid)

## FedAvgM Simulation

In [ ]:
# To reproduce Figure 5 from the Paper
from collections import defaultdict

fedavgm = defaultdict(list)

for CONCENTRATION in [100., 10., 1., 0.5, 0.2, 0.1, 0.05, 0.0]:
  for REPORTING_FRACTION in [0.05, 0.1, 0.4]:
    for LOCAL_EPOCHS in [1, 5]:
      print(f"\n\n>>> Running with the following parameters:\n>>>> Concentration (alpha): {CONCENTRATION}\n>>>> Reporting Fraction (C): {REPORTING_FRACTION}\n>>>> Local Epochs (E): {LOCAL_EPOCHS}\n\n")
      # Create FedAvgM strategy
      strategy_fedavgm = fl.server.strategy.FedAvgM(
          fraction_fit=REPORTING_FRACTION,
          fraction_evaluate=REPORTING_FRACTION,
          server_learning_rate=LEARNING_RATE,
          server_momentum=MOMENTUM,
          initial_parameters = fl.common.ndarrays_to_parameters(model.get_weights())
      )

      # Start simulation
      hist = fl.simulation.start_simulation(
          client_fn=client_fn,
          num_clients=NUM_CLIENTS,
          config=fl.server.ServerConfig(num_rounds=10), #COMMUNICATION_ROUNDS),
          strategy=strategy_fedavgm,
      )
      # TODO: Calculate Accuracy on the test set
      fedavgm[(CONCENTRATION, REPORTING_FRACTION, LOCAL_EPOCHS)].append(1)

INFO flwr 2023-08-22 20:14:13,348 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)




>>> Running with the following parameters:
>>>> Concentration (alpha): 100.0
>>>> Reporting Fraction (C): 0.05
>>>> Local Epochs (E): 1




2023-08-22 20:14:17,945	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-08-22 20:14:19,629 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 16252830105.0, 'memory': 32505660212.0, 'CPU': 8.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'object_store_memory': 16252830105.0, 'memory': 32505660212.0, 'CPU': 8.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0}
INFO flwr 2023-08-22 20:14:19,634 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-08-22 20:14:19,637 | server.py:269 | Using initial parameters provided by strategy
INFO:flwr:Using initial parameters provided by strategy
INFO flwr 2023-08-22 20:14:19,640 | server.py:88 | Evaluating initial parameters
INFO:flwr:Evaluating initial parameters
INFO flwr 2023-08-22 20:14:19,643 | server.py:101 | FL starting
INFO:flwr:FL starting
DEBUG flwr 2023-08-22 20:14:

(launch_and_fit pid=1947540) [!!!!!!!!!!] CONCENTRATION: 100.0 | LOCAL_EPOCHS: 1
(launch_and_fit pid=1947540) 15/15 - 2s - loss: 2.2919 - accuracy: 0.1067 - 2s/epoch - 118ms/step


DEBUG flwr 2023-08-22 20:14:39,632 | server.py:232 | fit_round 1 received 5 results and 0 failures
DEBUG:flwr:fit_round 1 received 5 results and 0 failures
WARNING flwr 2023-08-22 20:14:39,684 | fedavgm.py:214 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-08-22 20:14:39,687 | server.py:168 | evaluate_round 1: strategy sampled 5 clients (out of 100)
DEBUG:flwr:evaluate_round 1: strategy sampled 5 clients (out of 100)
(launch_and_evaluate pid=1947546) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 17 variables whereas the saved optimizer has 1 variables.  [repeated 5x across cluster]


...

(launch_and_fit pid=1965076) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 17 variables whereas the saved optimizer has 1 variables.  [repeated 4x across cluster]


(launch_and_fit pid=1965076) [!!!!!!!!!!] CONCENTRATION: 100.0 | LOCAL_EPOCHS: 1 [repeated 4x across cluster]
(launch_and_fit pid=1965082) 15/15 - 2s - loss: 2.2124 - accuracy: 0.1311 - 2s/epoch - 158ms/step [repeated 4x across cluster]
(launch_and_fit pid=1965079) [!!!!!!!!!!] CONCENTRATION: 100.0 | LOCAL_EPOCHS: 1 [repeated 7x across cluster]
(launch_and_fit pid=1965076) 15/15 - 3s - loss: 2.2099 - accuracy: 0.1800 - 3s/epoch - 217ms/step [repeated 2x across cluster]


(launch_and_fit pid=1965078) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 17 variables whereas the saved optimizer has 1 variables.  [repeated 8x across cluster]


(launch_and_fit pid=1965076) [!!!!!!!!!!] CONCENTRATION: 100.0 | LOCAL_EPOCHS: 1
(launch_and_fit pid=1965082) 15/15 - 4s - loss: 2.2342 - accuracy: 0.1511 - 4s/epoch - 260ms/step [repeated 6x across cluster]
(launch_and_fit pid=1965078) [!!!!!!!!!!] CONCENTRATION: 100.0 | LOCAL_EPOCHS: 1


DEBUG flwr 2023-08-22 20:52:55,552 | server.py:232 | fit_round 2 received 40 results and 0 failures
DEBUG:flwr:fit_round 2 received 40 results and 0 failures
DEBUG flwr 2023-08-22 20:52:55,762 | server.py:168 | evaluate_round 2: strategy sampled 40 clients (out of 100)
DEBUG:flwr:evaluate_round 2: strategy sampled 40 clients (out of 100)


## FedAvg Simulation

In [31]:
fedavg = defaultdict(list)

for CONCENTRATION in [100., 10., 1., 0.5, 0.2, 0.1, 0.05, 0.0]:
  for REPORTING_FRACTION in [0.05, 0.1, 0.4]:
    for LOCAL_EPOCHS in [1, 5]:
      print(f"\n\n>>> Running with the following parameters:\n>>>> Concentration (alpha): {CONCENTRATION}\n>>>> Reporting Fraction (C): {REPORTING_FRACTION}\n>>>> Local Epochs (E): {LOCAL_EPOCHS}\n\n")
      # Create FedAvg strategy
      strategy_fedavg = fl.server.strategy.FedAvg(
          fraction_fit=REPORTING_FRACTION,
          fraction_evaluate=REPORTING_FRACTION
      )

      # Start simulation
      hist = fl.simulation.start_simulation(
          client_fn=client_fn,
          num_clients=NUM_CLIENTS,
          config=fl.server.ServerConfig(num_rounds=10), #COMMUNICATION_ROUNDS),
          strategy=strategy_fedavg,
      )


      # TODO: Calculate Accuracy on the test set
      fedavg[(CONCENTRATION, REPORTING_FRACTION, LOCAL_EPOCHS)].append(1)

NameError: ignored

---
## FedAvgM and FedAvg performance curves for different non-identical-ness

In [ ]:
# TBD